In [1]:
# Import necessary packages:
from scipy import io
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# Data reading function:
def read_spam_data():
    raw_data = io.loadmat('/Users/shiyimin/Desktop/Grade3/CSC4020/hw2program/spamData.mat')
    # struct of raw_data is a dictionary:{'Xtrain':{}, 'ytrain':{}, 'Xtest':{}, 'ytest':{}}
    train_data = (raw_data['Xtrain'],raw_data['ytrain'])
    test_data = (raw_data['Xtest'],raw_data['ytest'])
    return train_data, test_data

In [3]:
# Declare objects in np.matrix data type to store the data
train_data, test_data = read_spam_data()
Xtrain = np.matrix(train_data[0])
ytrain = np.matrix(train_data[1])
Xtest = np.matrix(test_data[0])
ytest = np.matrix(test_data[1])
# data shape discovery:
print('Xtrain.shape: \t',Xtrain.shape)
print('ytrain.shape: \t',ytrain.shape)
print('Xtest.shape: \t',Xtest.shape)
print('ytest.shape: \t',ytest.shape)

Xtrain.shape: 	 (3065, 57)
ytrain.shape: 	 (3065, 1)
Xtest.shape: 	 (1536, 57)
ytest.shape: 	 (1536, 1)


We find the 3065 samples in traning set and 1536 samples in test set while each sample has 57 features.

In [6]:
# Data transformation functions
# function1: log
def transform_log(x):
    return np.log(x + 0.1)

# function2: binary
def transform_binary(x):
    # return 1 in int 32 when x>0
    return (x > 0).astype(np.int32)

# function3: standardlize
def transform_standard(x):
    return preprocessing.scale(x, axis=0)

# encapsulation
transform_functions = [transform_log,transform_binary,transform_standard]

Then for each version of the data,implement spam classiﬁcation using logistic regression and naive Bayes method, respectively.

In [7]:
# Fix the random seed for debug
np.random.seed(0)

For 8.2:
1. We construct the BinaryNaiveBayes Class whose input is binary discrete dataset.

In [9]:
class BinaryNaiveBayes:
    
    # data field:
    Xtrain = np.matrix([])
    ytrain = np.matrix([])
    Xtest = np.matrix([])
    ytest = np.matrix([])
    W = np.matrix([])
    N_pos = 0
    N_neg = 0
    N = 0
    
    
    def __init__(self, Xtrain, ytrain, Xtest, ytest, trans_f):
        self.Xtrain = np.matrix(trans_f(Xtrain))
        self.ytrain = ytrain.copy()
        self.Xtest = np.matrix(trans_f(Xtest))
        self.ytest = ytest.copy()
        
    def train(self):
        W_buffer = np.zeros([2,self.Xtrain.shape[1]])
        Xtrain_pos = self.Xtrain[np.array(ytrain==1).squeeze()]
        Xtrain_neg = self.Xtrain[np.array(ytrain==0).squeeze()]
        for i in range(int(Xtrain_pos.shape[0])):
            for d in range(int(Xtrain_pos.shape[1])):
                W_buffer[0][d] += Xtrain_pos[i,d]
        for j in range(int(Xtrain_neg.shape[0])):
            for d in range(int(Xtrain_neg.shape[1])):
                W_buffer[1][d] += Xtrain_neg[j,d]
        W_buffer[0] /=  Xtrain_pos.shape[0]
        W_buffer[1] /=  Xtrain_neg.shape[0]
        self.W = W_buffer
        self.N_pos = Xtrain_pos.shape[0]
        self.N_neg = Xtrain_neg.shape[0]
        self.N = Xtrain.shape[0]
    
    def single_forward(self, x):
        P_Class_pos = 0.0;
        P_Class_neg = 0.0;
        # firstly calculate the naive P(Class = 1| x) ~ P(x|1)*P(1)
        P_pos = self.N_pos/self.N
        P_x_when_pos = 1
        for d in range(x.shape[1]):
            P_x_when_pos *= self.W[0,d]**x[0,d]*(1-self.W[0,d])**(1-x[0,d])
        P_Class_pos = P_pos*P_x_when_pos
        # then calculate the naive P(Class = 0| x) ~ P(x|0)*P(0)
        P_neg = self.N_neg/self.N
        P_x_when_neg = 1
        for d in range(x.shape[1]):
            P_x_when_neg *= self.W[1,d]**x[0,d]*(1-self.W[1,d])**(1-x[0,d])
        P_Class_neg = P_neg*P_x_when_neg
        return int(P_Class_pos>P_Class_neg)
            
    def forward(self, X):
        y_ = np.zeros(X.shape[0])
        for i in range(y_.shape[0]):
            y_[i] = self.single_forward(X[i])
        return y_
    
    def test(self):
        y_ = self.forward(self.Xtest)
        correct_count = 0
        for i in range(len(self.ytest)):
            if y_[i] == self.ytest[i]: 
                correct_count += 1
        return correct_count/len(y_)
        
    
# Train with Xtrain,ytrain and test with Xtest,ytest
b = BinaryNaiveBayes(Xtrain, ytrain, Xtest, ytest, transform_functions[1])
b.train()
accuracy = b.test()
print('The accuracy for binary is:',str(accuracy))
print('The error rate for binary is:',str(1-accuracy))

The accuracy for binary is: 0.8899739583333334
The error rate for binary is: 0.11002604166666663
